An independent set or anticlique in a graph is a subset of the vertex set which spans no edges. A colouring is thus just a partition of the vertices into independent sets.

The complement of a graph $G$, denoted $G'$, has the same set of vertices as $G$. However, an edge exists between two vertices in $G'$ if and only if there is no edge between them in $G$.

A set of vertices that forms a clique in the complement graph $G'$ must be an anticlique in the original graph $G$. Therefore, to find a maximum independent set in $G$, we can simply find a maximum clique in its complement, $G'$.

The colouring procedure works as follows:
*   Given a graph $G$, find a largest possible independent set $I_1$. Since no two vertices in $I_1$ are connected, they can all be assigned the same colour $1$.
*   Remove the vertices of I_1 from the graph, creating a new, smaller graph $G' = G - I₁$. Find a largest independent set $I_2$ in $G'$. These vertices can all be assigned colour $2$.
*   Continue this process until all vertices have been removed (coloured). The total number of independent sets found is the total number of colours used.

At each step, the algorithm greedily tries to colour the largest possible number of vertices with a single new colour. This gives us a new upper bound for the chromatic number, $\chi(G)$.

In [5]:
import networkx as nx
import random
import numpy as np

def generate_Gknp(k, n, p):
    '''
    Generates a random graph G_k(n, p).
    '''
    G = nx.Graph()
    G.add_nodes_from(range(n))
    for i in range(n):
        for j in range(i + 1, n):
            if (j - i) % k != 0:
                if random.random() < p:
                    G.add_edge(i, j)
    return G

def generate_Gnp(n, p):
    '''
    Generates a random graph G(n, p).
    '''
    G = nx.Graph()
    G.add_nodes_from(range(n))
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < p:
                G.add_edge(i, j)
    return G

def greedy_colouring(graph, order):
    '''
    Applies the simple greedy colouring algorithm.
    '''
    colours = {}
    for vertex in order:
        neighbor_colours = {colours.get(neighbor) for neighbor in graph.neighbors(vertex)}
        colour = 1
        while colour in neighbor_colours:
            colour += 1
        colours[vertex] = colour
    return max(colours.values()) if colours else 0

def order_smallest_last(graph):
    '''
    Generates a vertex ordering using the smallest-last (degeneracy) heuristic.
    '''
    g_copy = graph.copy()
    ordering = []
    result_order = []
    while g_copy.number_of_nodes() > 0:
        min_degree_vertex = min(g_copy.nodes(), key=lambda v: g_copy.degree(v))
        result_order.append(min_degree_vertex)
        g_copy.remove_node(min_degree_vertex)
    return result_order[::-1]

def get_best_simple_greedy_colouring(graph):
    '''
    Runs the four simple greedy heuristics and returns the best result.
    '''
    if graph.number_of_nodes() == 0:
        return 0
    orders = [
        sorted(graph.nodes(), key=lambda v: graph.degree(v)),  # Increasing Degree
        sorted(graph.nodes(), key=lambda v: graph.degree(v), reverse=True),  # Decreasing Degree
        order_smallest_last(graph) , # Smallest Last
        random.sample(list(graph.nodes()), graph.number_of_nodes()) # Random
    ]

    min_colours = float('inf')
    for order in orders:
        if not order: continue
        num_colours = greedy_colouring(graph, order)
        if num_colours < min_colours:
            min_colours = num_colours

    return min_colours if min_colours != float('inf') else 0

def find_clique(graph):
    '''
    Finds a large clique using a repeated greedy search.
    '''
    nodes = list(graph.nodes())
    random.shuffle(nodes)
    best_clique_found = []
    for start_node in nodes:
        current_clique = [start_node]
        candidates = list(graph.neighbors(start_node))
        random.shuffle(candidates)
        for candidate_node in candidates:
            is_fully_connected = all(graph.has_edge(candidate_node, member) for member in current_clique)
            if is_fully_connected:
                current_clique.append(candidate_node)
        if len(current_clique) > len(best_clique_found):
            best_clique_found = current_clique
    return best_clique_found

In [6]:
def find_independent_set(graph):
    '''
    Finds a large independent set by finding a clique in the complement graph.
    '''
    if graph.number_of_nodes() == 0:
        return []
    complement_graph = nx.complement(graph)
    clique_in_complement = find_clique(complement_graph)
    return clique_in_complement

def colour_by_independent_sets(graph):
    '''
    colours a graph by repeatedly finding and removing the largest independent set.
    '''
    g_copy = graph.copy()
    num_colours = 0
    while g_copy.number_of_nodes() > 0:
        independent_set = find_independent_set(g_copy)
        if not independent_set:
            break
        g_copy.remove_nodes_from(independent_set)
        num_colours += 1
    return num_colours

In [7]:
print("=" * 42)
print("Comparing colouring methods on G_7(70, 0.5)")
print("=" * 42)

results_new_method = []
results_best_greedy = []

for i in range(10):
    G = generate_Gknp(k=7, n=70, p=0.5)

    new_bound = colour_by_independent_sets(G)
    best_greedy_bound = get_best_simple_greedy_colouring(G)

    results_new_method.append(new_bound)
    results_best_greedy.append(best_greedy_bound)

print(f"Average for G_7(70, 0.5):")
print(f"  - Independent Set Method: {np.mean(results_new_method):.2f} colours")
print(f"  - Best Simple Greedy:     {np.mean(results_best_greedy):.2f} colours")

Comparing colouring methods on G_7(70, 0.5)
Average for G_7(70, 0.5):
  - Independent Set Method: 8.00 colours
  - Best Simple Greedy:     11.70 colours


In [8]:
print("=" * 34)
print("Behavior as p changes for G(70, p)")
print("=" * 34)

probabilities = [0.4, 0.5, 0.6]
for p in probabilities:
    results_new = []
    results_greedy = []
    for i in range(5): # Run 5 trials per probability value
        G = generate_Gnp(n=70, p=p)
        new_bound = colour_by_independent_sets(G)
        best_greedy_bound = get_best_simple_greedy_colouring(G)
        results_new.append(new_bound)
        results_greedy.append(best_greedy_bound)

    print(f"Results for p = {p:.1f}:")
    print(f"  - Avg. Independent Set Method: {np.mean(results_new):.2f} colours")
    print(f"  - Avg. Best Simple Greedy:     {np.mean(results_greedy):.2f} colours")

Behavior as p changes for G(70, p)
Results for p = 0.4:
  - Avg. Independent Set Method: 11.40 colours
  - Avg. Best Simple Greedy:     12.60 colours
Results for p = 0.5:
  - Avg. Independent Set Method: 14.00 colours
  - Avg. Best Simple Greedy:     15.00 colours
Results for p = 0.6:
  - Avg. Independent Set Method: 16.80 colours
  - Avg. Best Simple Greedy:     17.60 colours


The new method of colouring by repeatedly finding the largest independent set consistently (but not always) outperforms the best of the simple greedy heuristics. This is because it takes a more global view at each step, trying to "satisfy" as many vertices as possible with a single colour.

As the edge probability p increases, the graph becomes denser. Both methods use more colours, which is expected as ddenser graphs have smaller independent sets and larger cliques, making them harder to colour. The independent set method continues to provide a better upper bound (fewer colours) than the simple greedy heuristics across all tested probabilities.

The performance gap is stable. The difference between the two methods stays roughly the same (around 1-2 colours). This suggests that the problem gets harder for both algorithms as $p$ increases and the relative effectiveness of the independent set method over the simple greedy methods does not change when the probability is changed.

Assume the input is a dense graph from $\mathcal{G}(n, 0.5)$, where the number of edges $m$ is of order $O(n^2)$.

1. Greedy colouring algorithm (with a given vertex order)

    The algorithm iterates through each of the $n$ vertices once. For each vertex, it must check the colours of all its already-coloured neighbors to find the smallest available colour.
    
    Each vertex has, on average, $(n-1)/2$ neighbors, which is $O(n)$. Checking the colours of $O(n)$ neighbors takes $O(n)$ time. Thus, the total time complexity it O(n^2).

2. Repeated greedy clique-finding procedure

    This procedure runs a simple greedy search n times, once starting from each vertex in the graph. A single greedy search starts with one vertex. It then iterates through the remaining $\sim n$ vertices. For each candidate vertex, it checks if it is connected to all members of the clique found so far.
    
    Let the size of the clique be $k$. The check takes $O(k)$ time. The clique size is typically around $k \approx 2*\log_2(n)$. The inner search takes $O(nk)$ time. The entire procedure repeats this $n$ times. The total time is $O(n^2\log n)$.

3. Independent set-finding procedure

    This procedure leverages the clique-finder. To build the complement $G'$, we must iterate through all $C(n, 2) = O(n^2)$ possible pairs of vertices and add an edge if one doesn't exist in the original graph $G$. This takes $O(n^2)$ time.

    We then run our $O(n^2 \log n)$ clique-finding procedure on this complement graph. The clique-finding step dominates the procedure so the total time is $O(n^2 \log n)$.

4. colouring by repeatedly finding independent sets

    In each iteration, the algorithm finds a large independent set, assigns it a new colour, and removes its vertices from the graph. The loop continues until the graph is empty.

    The number of iterations is the number of colours used which we denote as $C$, typically on the order of $O(n/\log n)$. Each iteration calls the independent set-finding proceedure, which takes $O(n'^2 \log n')$ time, where $n'$ is the number of vertices remaining. A crude upper bound can be estimated by assuming each step takes as long as the first one, $O(n^3)$.